In [1]:
# Imports

# Standard Python Data Science imports
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pytorch imports
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchtext.data.utils import get_tokenizer

In [2]:
# Partly taken from https://github.com/prakashpandey9/Text-Classification-Pytorch
class SelfAttention(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
        super(SelfAttention, self).__init__()

        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        output_size : 2 = (pos, neg)
        hidden_sie : Size of the hidden_state of the LSTM
        vocab_size : Size of the vocabulary containing unique words
        embedding_length : Embeddding dimension of GloVe word embeddings
        weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
        
        --------
        
        """

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.weights = weights

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        self.dropout = 0.8
        self.bilstm = nn.LSTM(embedding_length, hidden_size, dropout=self.dropout, bidirectional=True)
        # We will use da = 350, r = 30 & penalization_coeff = 1 
        # as per given in the self-attention original ICLR paper
        self.W_s1 = nn.Linear(2*hidden_size, 350)
        self.W_s2 = nn.Linear(350, 30)
        self.fc_layer = nn.Linear(30*2*hidden_size, 2000)
        self.label = nn.Linear(2000, output_size)

    def attention_net(self, lstm_output):

        """
        Now we will use self attention mechanism to produce a matrix 
        embedding of the input sentence in which every row represents an
        encoding of the input sentence but giving an attention to a 
        specific part of the sentence. We will use 30 such embedding of 
        the input sentence and then finally we will concatenate all the 30 
        sentence embedding vectors and connect it to a fully connected layer 
        of size 2000 which will be connected to the output layer of size 2 
        returning logits for our two classes i.e., pos & neg.
        
        Arguments
        ---------
        lstm_output = A tensor containing hidden states corresponding to each time step of the LSTM network.
        ---------
        Returns : Final Attention weight matrix for all the 30 different sentence embedding in which each of 30 embeddings give
                  attention to different parts of the input sentence.
        Tensor size : lstm_output.size() = (batch_size, num_seq, 2*hidden_size)
                      attn_weight_matrix.size() = (batch_size, 30, num_seq)
        """
        attn_weight_matrix = self.W_s2(torch.tanh(self.W_s1(lstm_output)))
        attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
        attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)

        return attn_weight_matrix

    def forward(self, input_sentences, batch_size=None, return_attn=False):

        """ 
        Parameters
        ----------
        input_sentence: input_sentence of shape = (batch_size, num_sequences)
        batch_size : default = None. 
        Used only for prediction on a single sentence after training (batch_size = 1)
        return_attn : bool determining whether to return attention layer activation 
                      default = False
        
        Returns
        -------
        Output of the linear layer containing logits for pos & neg class.
        Attention layer if the return_attn is set
        
        """

        input = self.word_embeddings(input_sentences)
        input = input.permute(1, 0, 2)
        if batch_size is None:
            h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())

        output, (h_n, c_n) = self.bilstm(input, (h_0, c_0))
        output = output.permute(1, 0, 2)
        # output.size() = (batch_size, num_seq, 2*hidden_size)
        # h_n.size() = (1, batch_size, hidden_size)
        # c_n.size() = (1, batch_size, hidden_size)
        attn_weight_matrix = self.attention_net(output)
        # attn_weight_matrix.size() = (batch_size, r, num_seq)
        # output.size() = (batch_size, num_seq, 2*hidden_size)
        hidden_matrix = torch.bmm(attn_weight_matrix, output)
        # hidden_matrix.size() = (batch_size, r, 2*hidden_size)
        # Let's now concatenate the hidden_matrix and connect it to the fully connected layer.
        fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
        logits = self.label(fc_out)
        # logits.size() = (batch_size, output_size)
        if not return_attn:
            return logits
        else:
            return logits, attn_weight_matrix

In [3]:
def read_embeddings(filename, vocab_size=10000, compute_avg=True):
    """
    Utility function, loads in the `vocab_size` most common embeddings from `filename`
  
    Arguments:
      - filename:     path to file
                      automatically infers correct embedding dimension from filename
      - vocab_size:   maximum number of embeddings to load
      - compute_avg:  bool to decide whether to comnpute the average word embedding,
                      which can be used as a <unk> embedding

      Returns 
      - embeddings:   torch.FloatTensor matrix of size (vocab_size x word_embedding_dim)
      - vocab:        dictionary mapping word (str) to index (int) in embedding matrix
    """

    # get the embedding size from the first embedding
    with open(filename, encoding="utf-8") as file:
        word_embedding_dim = len(file.readline().split(" ")) - 1

    vocab = {}
    inv_vocab = {}
    
    if compute_avg:
        # Add extra embedding for <unk> and <pad>
        # last index is <unk>, first index is <pad>
        embeddings = np.zeros((vocab_size + 2, word_embedding_dim))
    else:
         # Only add extra embedding for <pad>
         # first index is <pad>
        embeddings = np.zeros((vocab_size + 1, word_embedding_dim))
        

    with open(filename, encoding="utf-8") as file:
        for idx, line in enumerate(file):

            if idx >= vocab_size:
                break
            
            cols = line.rstrip().split(" ")
            val = np.array(cols[1:])
            word = cols[0]
            embeddings[idx + 1] = val
            vocab[word] = idx + 1
            inv_vocab[idx + 1] = word
    
    # Set <unk> embedding to the average of all other embedding vects in vocab
    if compute_avg:
        embeddings[-1] = embeddings[:-1].mean(axis=0)
    
    return torch.FloatTensor(embeddings), vocab, inv_vocab

In [4]:
def sent_embedd(sent, model, vocab_dict, tokenizer, max_length, device, unk_embedd=True):
    '''
    Helper function to extract the embedding for an input sentence.
    '''
    idxs = [0 for i in range(max_length)]
    i = 0
    for word in tokenizer(sent):
        if i < max_length:
            if word in vocab_dict:
                idxs[i] = vocab_dict[word]
            else:
                # If using <unk> embedding, append 
                # the final index where that embedding is stored
                if unk_embedd:
                    idxs[i] = len(vocab_dict)
            i += 1
    return torch.LongTensor([idxs]).to(device)
            
def sent_pred(sent, model, vocab_dict, tokenizer, max_length, device, batch_size):
    '''
    Runs the model on an input sentence.
    
    Arguments: 
    
      sent : str. The input sentence.
      model : the pytorch model to be used.
      vocab_dict : dict. A dictionary with words as keys and their indices as values
     
    Returns:
      pred : np array. The prediction, wich is a normalized array with a value for \
             each class, representing the predicted probability for that class
      attns : the attention matrix
    '''
    input_tensor = sent_embedd(sent, model, vocab_dict, tokenizer, max_length, device)
    input_tensor = torch.cat(batch_size * [input_tensor])
    pred, attns = model(input_tensor, return_attn=True)
    return pred.detach().cpu().numpy(), attns

In [5]:
def train_single(input_tensor_batch, target_tensor_batch, model, 
          model_optimizer, criterion):
    '''
    A single forward and backward pass of the neural net on a single training batch.
    '''
    target_tensor = torch.stack(target_tensor_batch).reshape(len(input_tensor_batch))
    input_tensor = torch.stack(input_tensor_batch)
    input_tensor = input_tensor.reshape(len(input_tensor_batch), input_tensor.shape[2])
    output = model(input_tensor, return_attn=False)
    loss = criterion(output, target_tensor)
    loss.backward()
    model_optimizer.step()
    return loss.item()

def train(input_tensors, target_tensors, input_val_tensors, target_val_tensors,
          model, model_optimizer, criterion, n_epochs):
    '''
    Train the attention classfier for a given number of epochs on the whole training set.
    '''
    losses = []
    train_accs = []
    val_accs = []
    # Iterate over given num of epochs
    for epoch in range(n_epochs):
        loss = 0
        # Iterate over batches
        for i in range(len(input_tensors)):
            input_tensor = input_tensors[i]
            target_tensor = target_tensors[i]
            loss += train_single(input_tensor, target_tensor, model, 
                                 model_optimizer, criterion)
        train_accuracy = get_accuracy(input_tensors, target_tensors, model)
        val_accuracy = get_accuracy(input_val_tensors, target_val_tensors, model)
        print(f"Epoch {epoch} :") 
        print(f"\tLoss {loss/len(input_tensors):.4f}")
        print(f"\tTraining Accuracy {train_accuracy:.4f}")
        print(f"\tValidation Accuracy {val_accuracy:.4f}")
        losses.append(loss/len(input_tensors))
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)
    return losses, train_accs, val_accs

def get_accuracy(input_tensors, target_tensors, model):
    '''
    Get model accuracy.
    '''
    accs = []
    # Iterate over batches
    for i in range(len(input_tensors)):
        input_tensor_batch = input_tensors[i]
        target_tensor_batch = target_tensors[i]
        target_tensor = torch.stack(target_tensor_batch).reshape(len(input_tensor_batch))
        input_tensor = torch.stack(input_tensor_batch)
        input_tensor = input_tensor.reshape(len(input_tensor_batch), input_tensor.shape[2])
        output = model(input_tensor, return_attn=False)
        # Get classification prediction
        preds = output.argmax(axis=1)
        # Get accuracy of given batch
        batch_acc = ((preds == target_tensor).sum()/target_tensor.shape[0]).item()
        accs.append(batch_acc)
    return np.mean(accs)

In [6]:
unk_embedd = True # Bool for whether to create an embedding for the <unk> token
# Get initial word vector embedings from disk
glove_filename = './GloveEmbeddings/glove.6B.100d.txt' 
glove_embeddings, vocab_dict, inv_vocab_dict = read_embeddings(glove_filename, 
                                                               vocab_size=100000,
                                                               compute_avg=unk_embedd)
# Create tokenizer
tokenizer = get_tokenizer('basic_english')

In [7]:
save_model = False # Whether to save model or not
load_model = False # Whether to load model or not
train_model = True # Whether to train model or not
checkpoint_path = "../models/attn_test.pth"
train_epochs = 14

In [8]:
if not load_model:
    # Set dimensions and hyperparameters
    batch_size = 32
    output_size = 2
    hidden_size = 100
    vocab_size = glove_embeddings.shape[0]
    embedding_length = glove_embeddings.shape[1]
    learning_rate = 5e-5
    max_length = 40 # max sentence length (in tokens)
    # Initialize embedding weights
    weights = glove_embeddings
    # Create model
    model = SelfAttention(batch_size, output_size, hidden_size, vocab_size, 
                          embedding_length, weights)
    #Define model optimizer
    model_optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    # Use cross entropy loss
    criterion = nn.CrossEntropyLoss()
    # Detect if we have a GPU available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
else:
    model = SelfAttention(batch_size, output_size, hidden_size, vocab_size, 
                      embedding_length, weights)
    checkpoint = torch.load(checkpoint_path)
    model_test.load_state_dict(checkpoint)

/home/dmac/Documents/MIMS Coursework/Capstone/env_marvin/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
# Senti tree bank
train_file = "../../../cross_style_transfer_internal/data/xslue/SentiTreeBank/train.tsv"
dev_file = "../../../cross_style_transfer_internal/data/xslue/SentiTreeBank/dev.tsv"
train_data = pd.read_csv(train_file, sep='\t')
val_data = pd.read_csv(dev_file, sep='\t')


# Short Humor Dataset:

# train_file = "../../../cross_style_transfer_internal/data/xslue/ShortHumor/train.tsv"
# dev_file = "../../../cross_style_transfer_internal/data/xslue/ShortHumor/dev.tsv"
# train_data = pd.read_csv(train_file, names=['domain', 'score', 'text'], sep='\t', error_bad_lines=False)
# val_data = pd.read_csv(dev_file, names=['domain', 'score', 'text'], sep='\t', 
#                        quoting=3, error_bad_lines=False)

# Politeness dataset:

# train_file = "../../../cross_style_transfer_internal/data/xslue/StanfordPoliteness/train.tsv"
# dev_file = "../../../cross_style_transfer_internal/data/xslue/StanfordPoliteness/dev.tsv"
# train_data = pd.read_csv(train_file, names=['domain', 'id', 'text', 'score'], sep='\t')
# val_data = pd.read_csv(dev_file, names=['domain', 'id', 'text', 'score'], sep='\t')

In [10]:
train_data['sentiment'].describe()

count    236076.000000
mean          0.513022
std           0.174044
min           0.000000
25%           0.416670
50%           0.500000
75%           0.611110
max           1.000000
Name: sentiment, dtype: float64

In [16]:
def parse_sentibank(senti_df):
    '''
    Parse SentiBank dataframe into the format we need for classification.
    '''
    input_df = pd.DataFrame()
    input_df['text'] = senti_df['phrase']
    # Map scores >= 0.5 (positive) to label 1 and scores < 0 (negative) to label 0.
    input_df['label'] = senti_df['sentiment'].apply(lambda x : int(x >= 0.5))
    return input_df


def parse_stanford_politeness(polite_df):
    '''
    Parse stanford politeness dataframe into the format we need for classification.
    '''
    input_df = pd.DataFrame()
    input_df['text'] = polite_df['text']
    # Map scores >= 0 (polite) to label 1 and scores < 0 (impolite) to label 0.
    input_df['label'] = polite_df['score'].apply(lambda x : int(x >= 0))
    return input_df

def df_to_training_pairs(df, batch_size, max_length):
    input_tensors = df['text'].apply(lambda x : sent_embedd(x, model, vocab_dict, tokenizer, max_length, device))
    target_tensors = df['label'].apply(lambda x : torch.LongTensor([x]).to(device))
    return input_tensors.values.reshape(-1, batch_size).tolist(), target_tensors.values.reshape(-1, batch_size).tolist()

In [12]:
train_df = parse_sentibank(train_data)
val_df = parse_sentibank(val_data)

In [13]:
train_df.head()

,text,label
0,! ',1
1,! '',1
2,! Alas,0
3,! Brilliant,1
4,! Brilliant !,1


In [14]:
# Testing a single pass on a single input

# sent = input_df['text'].iloc[0]
# target_tensor = torch.LongTensor([input_df['label'].iloc[0]]).to(device)
# input_tensor = sent_embedd(sent, model, vocab_dict)
# train_single(input_tensor, target_tensor, model, 
#           model_optimizer, criterion)

In [17]:
input_tensors, target_tensors = df_to_training_pairs(train_df.head((len(train_df)//batch_size)*batch_size), 
                                                                   batch_size, max_length)

input_val_tensors, target_val_tensors = df_to_training_pairs(val_df.head((len(val_df)//batch_size)*batch_size), 
                                                                   batch_size, max_length)

In [18]:
if train_model:
    train(input_tensors, target_tensors, input_val_tensors, target_val_tensors, 
          model, model_optimizer, criterion, train_epochs)

Epoch 0 :
	Loss 0.6848
	Training Accuracy 0.6348
	Validation Accuracy 0.5156
Epoch 1 :
	Loss 0.8102
	Training Accuracy 0.6346
	Validation Accuracy 0.5107
Epoch 2 :
	Loss 1.4141
	Training Accuracy 0.3681
	Validation Accuracy 0.4941
Epoch 3 :
	Loss 2.7998
	Training Accuracy 0.6351
	Validation Accuracy 0.5098
Epoch 4 :
	Loss 6.4531
	Training Accuracy 0.6336
	Validation Accuracy 0.5078
Epoch 5 :
	Loss 24.4944
	Training Accuracy 0.5512
	Validation Accuracy 0.5342
Epoch 6 :
	Loss 371.6418
	Training Accuracy 0.6319
	Validation Accuracy 0.5117
Epoch 7 :
	Loss 563400.5259
	Training Accuracy 0.5828
	Validation Accuracy 0.5049
Epoch 8 :
	Loss 922142530.7605
	Training Accuracy 0.3669
	Validation Accuracy 0.4902
Epoch 9 :
	Loss 43427386306833.8359
	Training Accuracy 0.6160
	Validation Accuracy 0.5215
Epoch 10 :
	Loss 659193539149840128.0000
	Training Accuracy 0.6134
	Validation Accuracy 0.5264
Epoch 11 :
	Loss nan
	Training Accuracy 0.3654
	Validation Accuracy 0.4902


KeyboardInterrupt: 

In [ ]:
if save_model:
    torch.save(model.state_dict(), checkpoint_path)
    with open('../models/vocab_dict.pickle', 'wb') as pickleFile:
        pickle.dump(vocab_dict, pickleFile)

To do:
- [X] handle unk words better
    - currently initalizing to the average of all word embeddings like suggested [here](https://stackoverflow.com/questions/49239941/what-is-unk-in-the-pretrained-glove-vector-files-e-g-glove-6b-50d-txt)
- [X] make work with batches
- [ ] use different, better embeddings
- [X] use better tokenizer, like spacy or some huggingface transformer model
- [ ] train and save a good model
- [ ] visualize attentions
- [ ] make work with other datasets
- [ ] convert to .py script that runs with input file that determines which data and parameters to use

In [ ]:
input_tensor_batch = input_tensors[0]
target_tensor_batch = target_tensors[0]
target_tensor = torch.stack(target_tensor_batch).reshape(len(input_tensor_batch))
input_tensor = torch.stack(input_tensor_batch)
input_tensor = input_tensor.reshape(len(input_tensor_batch), input_tensor.shape[2])
output, attention = model(input_tensor, return_attn=True)

In [ ]:
train_df.iloc[0]

In [ ]:
input_tensors[0][0]

In [ ]:
attention[0].shape

In [ ]:
tokenizer(train_df['text'].iloc[0])[attention[0].sum(axis=0).argmax().item()]

In [ ]:
attn = attention[0].sum(axis=0).cpu().detach().numpy()
plt.matshow([attn], cmap='Reds')

In [ ]:
sent = "It is raining."
tokens = tokenizer(sent)
output, attns = sent_pred(sent, model, vocab_dict, tokenizer, max_length, device, batch_size)
softmax = torch.nn.Softmax(dim=0)
pred_scores = softmax(torch.tensor(output[0]))
print(f"humorous : {pred_scores[0]*100:.2f}%, not : {pred_scores[1]*100:.2f}%")
attn, _ = attns[0].max(axis=0)
attn = attn.cpu().detach().numpy()
print(tokens)
plt.matshow([attn[:len(tokens)]], cmap='Reds');

In [ ]:
y = attn[:len(tokens)]
x = np.arange(len(tokens))
labels = tokens

fig, ax = plt.subplots(figsize=(10, 7))
plt.plot(x, y, 'bo')
ax.set_ylabel("Attention")
ax.set_xlabel("Token Position")

for i, txt in enumerate(labels):
    ax.annotate(txt, (x[i], y[i]), xytext=(x[i] + 0.015, y[i] + 0.015))
plt.savefig('Attention_funny.png')